In [ ]:
##########################
# compute FDR
# define FDR: numerator: the false positive alerts / denominator: the base alerts

# 1. FDR_org
# define base alert using less restricted rule: keep all alerts for FDR_org
#
# 2. FDR
# define base alert using more restricted rule: 
# keep those alerts with no-missing in its 4-hour prediction window for FDR
# drop those alerts with any length of missings, including a single missing
#

##########################
# compute leadtime

# define base alert using more restricted rule: 
# keep those alerts with no-missing in its 4-hour prediction window for FDR
# drop those alerts with any length of missings, including a single missing
# also drop those alerts entering the hypo episode with leadtime=0

# compute the interval between the alert and the first hypoSG (<70mgdL) ecountered in its 4-hr prediction window
# keep the first/earliest alert if multiple alerts have linked to the same hypoSG, drop the rest renewed ones

# define leadtime: the average of interval of all the kept alerts

##########################

# data source:
#run for every user based on the weekly production pull alert data is based on streams table, sg data is based on the feature vector table

##########################

# resulted summary statistics:

# person_id
# numb_total_alert - total number of alerts
# avg_alert_lag - average interval between two consecutive alerts
# stdv_alert_lag - standard deviation of alert interval
# numb_alert_missingSG - number of alerts having missing SG in the 4-hr window
# pct_missing_watchup - averge of percentage of missing SG in the 4-hr window
# numb_alert_non_missingSG - number of alerts having full 48 SGs in the 4-hr window, no missing
# numb_fp_alert_org - number of false positive alerts among all alerts - those ones don't get the hypo SG found in the 4-hr window
# FDR_org - FDR based on all alerts
# numb_fp_alert - number of false positive alerts among alerts with non-missingSG window - those ones don't get the hypo SG found in the 4-hr window
# FDR - FDR based on alerts with non-missingSG window
# nonrenew-lag - lagging intervals between two non-renewed alerts
# pct_renew_tp_alert - average of percentage of renewed true positive alerts - those ones have hypo SG found in the 4-hr window but it's the same hypo SG predicted by the previous or earlier alert
# numb_tp_alert - number of true positive alerts - excluding renewed alerts
# avg_leadtime - the average of the lead time among non-renewed alerts


##########################

In [19]:
import pandas as pd
import glob
import time
import os
import datetime
import numpy as np

src="/Users/lcao@us.ibm.com/Downloads/r3_production_analysis/input/"
dst="/Users/lcao@us.ibm.com/Downloads/r3_production_analysis/output/"


def compute_epoch_alert(s):
    return (int(time.mktime(time.strptime(s, "%Y-%m-%dT%H:%M:%SZ"))))

def compute_epoch_vector(s):
    a = int(time.mktime(time.strptime(s[0:19], "%Y-%m-%dT%H:%M:%S")))
    return (a - 3600*int(s[19:22]))

def check_watchup_drop_missing(e):
    pct_missing_watchup = np.nan
    has_missing=np.nan
    hypoSG_watchup=np.nan
    leadtime_watchup=np.nan
    first_hypoSG=np.nan
    epoch = e
    
    dt = dv[(dv['epoch']>=e) & (dv['epoch']<=(e+4*3600))]
    
    if(dt.shape[0]>0):
        has_missing = 0
        pct_missing_watchup=(1-dt.shape[0]/48)*100.
        if(pct_missing_watchup<0):
            pct_missing_watchup=0.
        if(pct_missing_watchup>0):
            has_missing = 1

        if(has_missing==0):  #only compute lead time for alert without SG missing in the watchup window
            
            if(dt['hypoSG'].sum()>0):
                hypoSG_watchup=1
                first_hypoSG = dt[dt['hypoSG']==1].reset_index()['epoch'][0]
                leadtime_watchup=(first_hypoSG-e)/60.
            else:
                hypoSG_watchup=0

    return ([epoch, has_missing, pct_missing_watchup,hypoSG_watchup,first_hypoSG,leadtime_watchup])


for filename in glob.iglob(src+"*_alert_db2.csv", recursive=True):
    print(filename)
    person_id=filename.replace(src,'').replace("_alert_db2.csv",'')
    da = pd.read_csv(src+person_id+"_alert_db2.csv",dtype=object)
    da = da[['userid','alert_start']]
    os.environ['TZ']='UTC' 
    da['epoch']=da['alert_start'].apply(compute_epoch_alert)
    da = da[['userid','epoch']]
    da['alert']=1
    da.columns.values[0] = "person_id"
    da.sort_values(['epoch'], ascending=[True])
    
    numb_total_alert=0
    avg_alert_lag=0
    stdv_alert_lag=0
    da['lag']=(da['epoch']-da['epoch'].shift(1))/60.
    da.loc[0,'lag']=240
    numb_total_alert=da.shape[0]
    avg_alert_lag=da['lag'].mean()
    stdv_alert_lag=da['lag'].std()
    print ('numb_total_alert=',numb_total_alert,'avg_alert_lag=',avg_alert_lag,'stdv_alert_lag=',stdv_alert_lag)
    if(numb_total_alert>0):
        #merge with feature vector to check the missing and hypoSG
        #hypo-SG in 4-hr watch-up window after the alert being sent
        dv = pd.read_csv(src+person_id+"_hypofeature_kafka.csv",dtype=object)
        dv = dv[['userid','doNotPublish','doNotScore','sg_timestamp','hypo','dayofweek','hourofday','sglatest']]
        dv['epoch']=dv['sg_timestamp'].apply(compute_epoch_vector)
        dv.columns.values[0] = "person_id"
        dv['hypoSG'] = np.where(pd.to_numeric(dv['sglatest'])>=80, 0, 1) #change to 70mgdL for more restricted rule
        dv.sort_values(['epoch'], ascending=[True])
        
        #loop through each eligible alert
        dd = da['epoch'].apply(check_watchup_drop_missing)
        k= pd.DataFrame(dd.values.tolist(), index=dd.index)
        k.columns = ['epoch','has_missing','pct_missing_watchup','hypoSG_watchup','first_hypoSG','leadtime_watchup']

        pct_missing_watchup=k['pct_missing_watchup'].mean()
        numb_alert_missingSG=len(k[k['pct_missing_watchup']>0].index)
        print ('pct_missing_watchup=',pct_missing_watchup)
        print ('numb_alert_missingSG=',numb_alert_missingSG)

        dh = pd.merge(da,k,on=['epoch'],how='left')

        #count the true negative alerts among alert having non-missing in watchup window
        #dh.groupby('has_missing').size()
        numb_alert_non_missingSG=len(dh[dh['has_missing']==0])
        #less restricted rule to include missing SG cases
        numb_fp_alert_org=len(dh[dh['hypoSG_watchup']==0])
        FDR_org=numb_fp_alert_org/len(dh)*100
        #more restricted rule to exclude missing SG cases
        numb_fp_alert=len(dh[(dh['has_missing']==0) & (dh['hypoSG_watchup']==0)])
        FDR=numb_fp_alert/numb_alert_non_missingSG*100
        print ('numb_fp_alert_org=',numb_fp_alert_org,'numb_fp_alert=',numb_fp_alert,'numb_alert_non_missingSG=',numb_alert_non_missingSG, 'FDR_org=',FDR_org, 'FDR=',FDR)

        #remove any alerts which is the hypoSG itself - its leadtime is 0
        dk = dh[dh['leadtime_watchup']!=0]

        #keep those ones having hypoSG found
        dk1 = dk[dk['hypoSG_watchup']==1].reset_index()


        #sort by first_hypoSG, and keep the earliest alert with the same first_hypoSG
        pct_renew_tp_alert=np.nan
        nonrenew_lag=np.nan
        sum_leadtime=np.nan
        numb_tp_alert=np.nan
        avg_leadtime=np.nan
        
        if(dk1.shape[0]>0):
            dk1['nonrenewlag']=(dk1['epoch']-dk1['epoch'].shift(1))/60.
            dk1.loc[0,'nonrenewlag']=240
            nonrenew_lag = dk1['nonrenewlag'].mean()
            print('nonrenew_lag=',nonrenew_lag)

            dk2 = dk1[['person_id','epoch','first_hypoSG','leadtime_watchup','hypoSG_watchup']].groupby('first_hypoSG').first().reset_index()

            #renew alert among true positive alerts
            pct_renew_tp_alert = (1-len(dk2)/len(dk1))*100
            print('pct_renew_tp_alert=',pct_renew_tp_alert)

            numb_tp_alert=len(dk2)
            if(numb_tp_alert>0):
                sum_leadtime=dk2['leadtime_watchup'].sum()
                avg_leadtime=sum_leadtime/numb_tp_alert
                print('avg_leadtime=',avg_leadtime,'numb_tp_alert=',numb_tp_alert)
                
    #print(person_id,numb_total_alert,avg_alert_lag,stdv_alert_lag,numb_alert_missingSG,pct_missing_watchup, \
          #numb_alert_non_missingSG,numb_fp_alert_org,FDR_org,numb_fp_alert,FDR, \
          #nonrenew_lag,pct_renew_tp_alert,numb_tp_alert,avg_leadtime)  
            
    dr = pd.DataFrame([[person_id,numb_total_alert,avg_alert_lag,stdv_alert_lag,numb_alert_missingSG,pct_missing_watchup, \
          numb_alert_non_missingSG,numb_fp_alert_org,FDR_org,numb_fp_alert,FDR, \
          nonrenew_lag,pct_renew_tp_alert,numb_tp_alert,avg_leadtime]],
                      columns=['person_id','numb_total_alert','avg_alert_lag','stdv_alert_lag','numb_alert_missingSG','pct_missing_watchup', \
          'numb_alert_non_missingSG','numb_fp_alert_org','FDR_org','numb_fp_alert','FDR', \
          'nonrenew_lag','pct_renew_tp_alert','numb_tp_alert','avg_leadtime'])
    print(dr)
    dr.to_csv(dst+person_id+"_leadtime_alternative.csv", index=False, encoding='utf-8')
    
    

/Users/lcao@us.ibm.com/Downloads/r3_production_analysis/input/50788056_alert_db2.csv
numb_total_alert= 20 avg_alert_lag= 324.255 stdv_alert_lag= 420.7006834572574
pct_missing_watchup= 8.645833333333334
numb_alert_missingSG= 4
numb_fp_alert_org= 4 numb_fp_alert= 4 numb_alert_non_missingSG= 16 FDR_org= 20.0 FDR= 25.0
nonrenew_lag= 649.3854166666666
pct_renew_tp_alert= 25.0
avg_leadtime= 64.16388888888889 numb_tp_alert= 6
  person_id  numb_total_alert  avg_alert_lag  stdv_alert_lag  \
0  50788056                20        324.255      420.700683   

   numb_alert_missingSG  pct_missing_watchup  numb_alert_non_missingSG  \
0                     4             8.645833                        16   

   numb_fp_alert_org  FDR_org  numb_fp_alert   FDR  nonrenew_lag  \
0                  4     20.0              4  25.0    649.385417   

   pct_renew_tp_alert  numb_tp_alert  avg_leadtime  
0                25.0              6     64.163889  


In [12]:
import numpy as np
a = np.random.randn(4, 3) # a.shape = (4, 3)
b = np.random.randn(3, 2) # b.shape = (3, 2)
c = a*b
print(c.shape)

ValueError: operands could not be broadcast together with shapes (4,3) (3,2) 